# **`Flight Rover Post-Vibe Checkout`**

**Default Settings:**
\
`wired_seq_num = 0x01`
\
`itr=1`
\
`send_data_packet_to_wd_before_sniffing = True`
\
`specific_cmd_name_override = None`

**To sniff Serial from SBC without sending a command:**
1. `send_data_packet_to_wd_before_sniffing = False`

**To enter SERVICE mode:**
1. `itr = 1`

**To enter MISSION mode:**
1. `itr = 2`

**System Soft Reset:**  *(All systems except the WatchDog powered off)*
1. Enter **`SERVICE`** mode twice

**To Reset Specific Systems:**
1. specific_cmd_name_override = 'power-off-[system]`   
    *System can be one of the following:*
    - `herc`
    - `radio`
    - `motors`
    - `cameras`

**No telem from Herc after 200 sec**
1. `specific_cmd_name_override = 'power-off-herc'`
2. `specific_cmd_name_override = 'power-off-radio'`
3. `specific_cmd_name_override = 'power-on-radio'`
4. `specific_cmd_name_override = 'power-on-herc'`

**Misc Commands:**


## Additional Support:
See ***Pre-prepared commands list in `__command_aliases.py`***.

Logs of everything printed to the console will stored in `./raw-console-logs/` as `*.clog` test files under the name used for `settings['SAVE_FILE_PREFIX']`.

If you want to explore the Data Standards to build new commands, run:

`pyenv exec python datastandards_lookup.py`.

In [1]:
## SETUP:

from scripts.utils.trans_tools import *
from scripts.utils.__command_aliases import get_command
from IrisBackendv3.utils.basic import bytearray_to_spaced_hex as hexstr

seq_num = 0x01
blimp_cmds = ['setup', 'power-on', 'SetChargerEn', 'SetChargerPowerConnection', 'SetBatteryConnection', 'SetBatteryControlEnable', 'SetBatteryLatch']
cmd = blimp_cmds[4]
blimp_inputs = [False, True, 'OFF', 'ON', 'FORCE_HIGH', 'FORCE_LOW']
param = blimp_inputs[1]

# cmd, param = 'stop-heating', False

send_data_packet_to_wd_before_sniffing = True
specific_cmd_name_override = cmd

## CE Test One - All Internal Rover Systems Pre-Deployment Enabled
# specific_cmd_name_override = 'setup' # ~8         32
# specific_cmd_name_override = 'power-on' # ~14.5   38

# specific_cmd_name_override = '3v3-off' #    
# specific_cmd_name_override = '3v3-on' # also makes sure VSA is on too.   

# specific_cmd_name_override = 'power-off-herc' #    
# specific_cmd_name_override = 'power-off-radio' # 
# specific_cmd_name_override = 'power-on-herc' #    44
# specific_cmd_name_override = 'power-on-radio' #   45

# specific_cmd_name_override = 'reset-radio'
# specific_cmd_name_override = 'reset-herc' #         59 -> 63
## FAILED

# specific_cmd_name_override = 'SetBatteryConnection'
# param = False

## CE Test One - All External Rover Systems Post-Deployment Enabled
## CE Test Two - All Rover Systems Enabled
## CE Test Three - 

# specific_cmd_name_override = 'set-CE'

# REGE_set = False
# BE_set = False
# CE_set = 'OFF' # 'OFF' 'ON' 'FORCE_HIGH'
# BCE_set = 'OFF' # 'OFF' 'ON' 'FORCE_HIGH'
# BL_set = 'OFF' # 'OFF' 'ON' 'PULSE_HIGH'


print(specific_cmd_name_override,'  :  ',param)

SetBatteryConnection   :   True


In [2]:
## SETTINGS:

serial_device = '/dev/tty.usbserial-A7035PDL'
ip="192.168.1.2"
port=8080

settings['SAVE_FILE_PREFIX'] = 'iris__flight_post_vibe__WD_prog_attempt__11_21_21F' # this is the prefix on all log files. make it something unique.
load_cache()
setup_logger(settings['SAVE_FILE_PREFIX']) # logs of stuff printed to console will be stored to `raw-console-logs`

# IF ERROR - update last number in FILE_PREFIX

In [3]:
step = specific_cmd_name_override
pathway, magic, command_name, kwargs, telem_pathway = get_command(step, param)
print(pathway, magic, command_name, kwargs)

packet = build_command_packet(seq_num, pathway, magic, command_name, kwargs)
packet_bytes = packet.encode()
print(hexstr(packet_bytes))

DataPathway.WIRED Magic.WATCHDOG_COMMAND WatchDogInterface_SetBatteryConnection {'batt_en': True}
01 07 00 42 ee ff 00 c0 f9 10 ff


In [4]:
## Send Command:
connect_serial(device = serial_device)
if send_data_packet_to_wd_before_sniffing:
    send_packet(packet, pathway, ip, port)

Connection Success!


In [5]:
## Stream Telemetry:
stream_data_ip_udp_serial()
# Check for wireless telemetry in Wireshark. For parsing help, run: `pyenv exec python parse_pcap.py --help`

2021-11-22 00:32:02: INFO     > Command[#249: WatchDogInterface_SetBatteryConnection] -> NO_ERROR[0x0]
2021-11-22 00:32:04: INFO     [HEAT]: OFF 		 [BATT]: Voltage: BAD, Charge: 1880mAh =  53.7%, Temp:  75.0°K]
2021-11-22 00:32:06: INFO     #215 [RS_KEEP_ALIVE + NOT_DEPLOYED] 	CHARGER: OFF 	BATTERIES: TEMP_CONNECTED 	UART0: OFF 	UART1: INITIALIZED_&_ACTIVE
1V2: BAD 	1V8: BAD 	2V5: 0.0 	2V8: 0.0 	3V3: BAD 	5V0: BAD 	24V: 0.41
VBm: 14.87V 	VL: 24.07V 	Vcc28: 26.10V 	VSA: 0.99V 	ISA: 3.7mA
HEATER [268K |274K |273K] 	 is HEATING, Control: DISABLED 	@ 0.000W / 0.704W 	Period: 1.2ms
Tbatt: 428.1K ± 0.0K 	Tchrg: 0.0K ± 0.0K 	Tblimp: 0.0K
BATTERY MONITOR: 0.00V 	0.0mA 	0mAh 	
       2V5    2V8    24V     VBm      VL   Vcc28    VSA    ISA       Tbatt     Tchrg
ADC      0      0     64    2224    3104    2944    128      1           1         0
val  0.00V  0.00V  0.41V  14.87V  24.07V  26.10V  0.99V  3.7mA  428.1±0.0K  0.0±0.0K
              HEAT  DEP  BSTAT  BE  LB  LS  LR  CE  REGE  VSAE  3V3_